In [9]:

"""
Using MNIST, compare classification performance of:
1) logistic regression by itself,
2) logistic regression on outputs of an RBM, and
3) logistic regression on outputs of a stacks of RBMs / a DBN.
"""

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [10]:
def norm(arr):
    arr = arr.astype(np.float)
    arr -= arr.min()
    arr /= arr.max()
    return arr

In [12]:
# load MNIST data set
mnist = load_digits()
X, Y = mnist.data, mnist.target

# normalize inputs to 0-1 range
X = norm(X)

# split into train, validation, and test data sets
X_train, X_test, Y_train, Y_test = train_test_split(X,       Y,       test_size=200, random_state=0)
labels = np.random.binomial(n=1, p=0.5, size=[1000]).reshape(1000,1)
data = np.random.rand(1000, 64)
data = norm(data)

In [ ]:
# --------------------------------------------------------------------------------
# set hyperparameters

learning_rate = 0.02 # from Erhan et el. (2010): median value in grid-search
total_units   =  800 # from Erhan et el. (2010): optimal for MNIST / only slightly worse than 1200 units when using InfiniteMNIST
total_epochs  =   50 # from Erhan et el. (2010): optimal for MNIST
batch_size    =  128 # seems like a representative sample; backprop literature often uses 256 or 512 samples

C = 100. # optimum for benchmark model according to sklearn docs: https://scikit-learn.org/stable/auto_examples/neural_networks/plot_rbm_logistic_classification.html#sphx-glr-auto-examples-neural-networks-plot-rbm-logistic-classification-py)

# TODO optimize using grid search, etc

# --------------------------------------------------------------------------------
# construct models

# RBM
rbm1 = BernoulliRBM(n_components=total_units, learning_rate=learning_rate, batch_size=batch_size, n_iter=total_epochs, verbose=1)
rbm2 = BernoulliRBM(n_components=int(total_units/2), learning_rate=learning_rate, batch_size=batch_size, n_iter=total_epochs, verbose=1)
rbm3 = BernoulliRBM(n_components=int(total_units/4), learning_rate=learning_rate, batch_size=batch_size, n_iter=total_epochs, verbose=1)


models = []                       
model = Pipeline(steps=[('rbm1', clone(rbm1)), ('rbm2', clone(rbm2)),('rbm3', clone(rbm3))])  # RBM stack / DBN
# --------------------------------------------------------------------------------
# train and evaluate models

model.fit(X_train)

def transform_through_dbn(data, rbms):
    for rbm in rbms:
        data = rbm.transform(data)
    return data
    
# Extract features using DBN
dbn_features = transform_through_dbn(X_train, model)  # Replace with your DBN layers

# Initialize and train the classifier
classifier = LogisticRegression()
classifier.fit(dbn_features, Y_train)

# Predict and evaluate (assuming you have a test set)
predictions = classifier.predict(transform_through_dbn(X_train, model))
# Evaluate predictions using appropriate metrics

In [17]:
from joblib import dump

dump(model, 'model.joblib')

['model.joblib']

In [ ]:
from joblib import load

dbn_pipeline = load('model.joblib')

In [ ]:
indices = ["0", "1", "10", "11", "12", "13"]
data_S = []
for index in indices:
    data = np.load(f"../../data/data{index}.npy")
    data_S.append(data)

# Assuming 'data' is your original (291, 275, 442) array
train = []
for data in data_S:
    new = []
    for i in range(442):
        # Isolate one channel (for example, the first channel)
        channel_data = data[:, :, i]  # Shape will be (291, 275)

        # Flatten the channel data
        channel_data_flattened = channel_data.reshape(-1)  # Flatten into a vector

        # Normalize
        scaler = StandardScaler()
        channel_data_normalized = scaler.fit_transform(channel_data)
        #channel_data_normalized = channel_data_normalized.ravel()
        new.append(channel_data_normalized)
    train.append(np.array(new).T)

In [ ]:
indices = ["0", "1", "10", "11", "12", "13"]
data_S = []
for index in indices:
    data = np.load(f"../../data/data{index}.npy")
    data_S.append(data)

# Assuming 'data' is your original (291, 275, 442) array
train = []

new = []
for i in range(442):
    # Isolate one channel (for example, the first channel)
    channel_data = data[:, :, 0]  # Shape will be (291, 275)

    # Flatten the channel data
    channel_data_flattened = channel_data.reshape(-1)  # Flatten into a vector

    # Normalize
    scaler = StandardScaler()
    channel_data_normalized = scaler.fit_transform(channel_data)
    arr = np.array(channel_data_normalized)  # Replace with your array
    rows, cols = arr.shape

    # Trimming to make square
    if rows > cols:
        square_arr = arr[:cols, :]
    elif cols > rows:
        square_arr = arr[:, :rows]

    #channel_data_normalized = channel_data_normalized.ravel()
    new.append(square_arr)
    